In [2]:
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, Z1, pairing, neg
import numpy as np
import galois   

In [3]:
GF = galois.GF(curve_order)
# takes about 2 minutes

In [5]:
x = GF(5)#random.randint(1,10)
y = GF(10)#random.randint(1,15)

v1 = x * x
v2 = v1 * x
v3 = y * y
v4 = v2 * y
out = v2 + GF(2) * v4 - GF(5) * x * v3 - GF(3) * y + GF(2)
print(1,out,x,y,v1,v2,v3,v4)

Lgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,curve_order - 5,0,0,0,0,0] # 113 - 5 = 108
    ]))

Rgf = GF(np.array([
    [0,0,1,0,0,0,0,0],
    [0,0,1,0,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,1,0,0,0,0],
    [0,0,0,0,0,0,1,0]
    ]))

Ogf = GF(np.array([
    [0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,0,0],
    [0,0,0,0,0,0,1,0],
    [0,0,0,0,0,0,0,1],
    [curve_order-2,1,0,3,0,curve_order-1,0,curve_order-2] # 113 - 2 = 111
    ]))

wgf = GF(np.array([GF(1),out,x,y,v1,v2,v3,v4]))

# assert all(np.multiply(np.dot(L,w), np.dot(R,w)) == np.dot(O,w)), "result contains an inequality"
result = Ogf.dot(wgf) == np.multiply(Lgf.dot(wgf),Rgf.dot(wgf))
assert result.all(), "result contains an inequality"


1 97 5 10 25 125 100 1250


In [6]:
target_poly = GF(np.array([1,2,3,4,5]))
# for each column in the matrix, compute the lagrange polynomial, resulting in a vector of polynomials
def interpolate_columns(column):
    return galois.lagrange_poly(target_poly,column)

L_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Lgf)
R_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Rgf)
O_vector_of_poly = np.apply_along_axis(interpolate_columns, 0, Ogf)
# print(L_vector_of_poly)

In [7]:
def reduce_polynomials_to_1(vector_of_poly, witness):
    sum = GF(0)
    for i in range(len(vector_of_poly)):
        sum += vector_of_poly[i]*witness[i]
    return sum

Ux = reduce_polynomials_to_1(L_vector_of_poly,wgf)
Vx = reduce_polynomials_to_1(R_vector_of_poly,wgf)
Wx = reduce_polynomials_to_1(O_vector_of_poly,wgf)
print(Ux)
print(Vx)
print(Wx)


7296080957279758407415468581752425029516121466805344781232734728858602831849x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301416197x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301414937x^2 + 7296080957279758407415468581752425029516121466805344781232734728858602833414x + 21888242871839275222246405745257275088548364400416034343698204186575808494842
13680151794899547013904003590785796930342727750260021464811377616609880309765x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253539795x^3 + 8208091076939728208342402154471478158205636650156012878886826569965928186022x^2 + 12768141675239577212977070018066743801653212566909353367157285775502554955532x + 125
6384070837619788606488535009033371900826606283454676683578642887751277477581x^4 + 9120101196599698009269335727190531286895151833506680976540918411073253543130x^3 + 15504172034219486615757870736223903187721758116961357660119561298824531005611x^2 + 12768141675239577

In [8]:
UxVx = Ux*Vx
Tx = galois.Poly(np.flip(np.polynomial.polynomial.polyfromroots([1,2,3,4])).astype(int),  field=GF)
Hx, remainder = divmod(((Ux*Vx) - Wx), Tx)
assert remainder == 0, "remainder is not 0"

# Check Tx and Hx balances the polynomial correctly
lhs = Ux*Vx
rhs = Hx*Tx + Wx
assert lhs == rhs, "result is not balanced across LHS and RHS"
print(lhs.degree)
print(rhs.degree)

8
8


In [15]:
# Compute Generator points and add together
# Uses py_ecc to scale both G1 and G2 points
from functools import reduce
import random

tau = GF(8)
# tau = GF(random.randint(1,curve_order-1))


def generate_powers_of_tau_G1(tau, degree):
    """
    tau is a random num between 1 and curve_order, degree is the highest d of the polynomial
    this generates the powers of tau, i.e. tau^0, tau^1, tau^2, tau^3, tau^4, tau^5
    """
    return [multiply(G1, int(tau ** i)) for i in range(degree + 1)]

def generate_powers_of_tau_G2(tau, degree):
    """
    same for G2
    """
    return [multiply(G2, int(tau ** i)) for i in range(degree + 1)]

def inner_product(ec_points, coeffs):
    """
    ec_points is the powers of tao, i.e. [G1, tau*G1, tau^2*G1, tau^3*G1, ...]
    coeffs are the coefficients of the polynomial after lagrange
    this computes the inner product of the 2 vectors i.e. Ux[0] * tau^0 + Ux[1] + tau^1..
    the result is 1 point on the curve representing the whole polynomial
    Z1 is the identity element
    """
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(ec_points, coeffs)), Z1)

# evaluate then convert
powers_of_tau_G1 = generate_powers_of_tau_G1(tau, 2 * Tx.degree)
powers_of_tau_G2 = generate_powers_of_tau_G2(tau, 2 * Tx.degree)

A_1 = inner_product(powers_of_tau_G1, Ux.coeffs[::-1])
B_2 = inner_product(powers_of_tau_G2, Vx.coeffs[::-1])
C_1prime = inner_product(powers_of_tau_G1, Wx.coeffs[::-1])

# Trying 2 different ways to compute HxTx
# Computing HxTx using the inner product of both Hx and Tx
# This doesn't work because the result HxTx is 2 points and needs to be added together
# We need this to be multiplied together
Hx_1 = inner_product(powers_of_tau_G1, Hx.coeffs[::-1])
Tx_1 = inner_product(powers_of_tau_G1, Tx.coeffs[::-1])
print(Hx_1, Tx_1)
HxTx = add(Hx_1, Tx_1)



# Option 2 - multiply polynomials together then compute inner product
HxTx_Alt = Hx * Tx
HxTx_Alt_1 = inner_product(powers_of_tau_G1, HxTx_Alt.coeffs[::-1])
print(HxTx_Alt_1)

C_1 = add(C_1prime, HxTx) # Invalid
C_1_1 = add(C_1prime, HxTx_Alt_1) # Valid

if(eq(pairing(B_2, A_1), pairing(G2, C_1_1))):
    print("true")
else:
    print("false")

(7534403868836795831123372764670274379169220370678040165966019577044802615912, 15377403639978586137353473008813821155232647661540644645717096409770515680086) (15866823055351122078559837406709798344525908146961222093235442230315196349165, 5221542821457984485577786120518539672849655396533551547116902579546982121924)
(11452539570018324317000093982080546782395866108619572977690869795141496834597, 3660237031297579395866883809407692653641847663246608659962846630882244473967)
(19186923886830265421807917454139781085750311457556253599382340809269003208297, 15407345950783423031291582780789281566364258675422988754415284882637160770622)
true
